In [1]:
#!pip install plyfile

In [2]:
#!unzip PointCNN_Pytorch.zip

In [3]:
from __future__ import print_function
import argparse
import os
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from data_class import ModelNet40,modelnet40_download
from transforms_3d import *


from pointcnn import PointCNNCls

from utils import progress_bar, adjust_lr_steep, log_row

In [4]:
data_,model_,name_="ModelNet40", "PointCNN", "cls"

In [5]:
def save_ckpt(epoch, model, optimizer, acc_list):
    
    if not os.path.isdir('checkpoints'):
        os.mkdir('checkpoints')
    
    if not os.path.isdir('checkpoints/%s_%s_%s'%(data_,model_,name_)):
        os.mkdir('checkpoints/%s_%s_%s'%(data_,model_,name_))
    if (epoch % 20 == 0) or (epoch in adj_lr['steps']) or (acc_list[-1] > max(acc_list[:-1])):
        print('=====> Saving checkpoint...')
        state = {
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'acc_list': acc_list,
                }
        torch.save(state, 'checkpoints/%s_%s_%s/epoch_%d.pth' % (data_,model_,name_, epoch))
        print('Successfully save checkpoint at epoch %d' % epoch)


In [6]:
adj_lr = {'steps' : [int(temp) for temp in [50, 80, 120, 150]], 
                   'decay_rates' : [float(temp) for temp in [0.1, 0.1, 0.2, 0.2]]}

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
num_classes = 40
model = PointCNNCls(num_classes)
model = model.to(device) 
model = nn.DataParallel(model)   

In [8]:
print('=====> Building new model...')
torch.manual_seed(0)
print("Random Seed: ", 0)       


optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
START_EPOCH = 0
acc_list = [0]
print('Successfully built!')

=====> Building new model...
Random Seed:  0
Successfully built!


In [9]:
train_tfs = compose([rotate_y(), 
                             rand_scale(), 
                             rand_translate(), 
                             jitter(), 
                             normalize()
                            ])
test_tfs = normalize()
train_data = ModelNet40(partition='train', num_points=2048, transforms=train_tfs)
test_data = ModelNet40(partition='test', num_points=2048, transforms=test_tfs)

/content/data_class.py:159: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  f = h5py.File(h5_name)


In [10]:
train_loader = DataLoader(train_data, batch_size=16, shuffle=True, drop_last=True)
test_loader = DataLoader(test_data, batch_size=16, shuffle=True, drop_last=False)
print('======> Successfully loaded!')

======> Successfully loaded!


In [11]:
criterion = F.cross_entropy

In [12]:
def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    loss_total=0
    for j, data in enumerate(test_loader, 0):
        points, label = data
        points, label = points.to(device), label.to(device)[:,0]
            
        
        points = points.transpose(2, 1) # to be shape batch_size*3*N

            
        pred, trans_feat = model(points)
                  
        loss = criterion(pred, label)
            
        pred_choice = pred.data.max(1)[1]
        correct += pred_choice.eq(label.data).cpu().sum()
        total += label.size(0)
        loss_total+=loss.item()
        pbar.set_postfix({'step':i+1,'Test Loss:' : total_loss/(i+1),'Test Acc':100.*correct.item()/total})
        
        
    
    return loss_total/(j+1), 100.*correct.item()/total

In [13]:
if not os.path.isdir('logs_train'):
        os.mkdir('logs_train')
logname = ('logs_train/%s_%s_%s.csv' % (data_,model_,name_))    
    
if os.path.exists(logname):
        with open(logname, 'a') as logfile:
            log_row(logname, [''])
            log_row(logname, [''])

In [23]:

from tqdm.notebook import tqdm
model.train()
for epoch in range(START_EPOCH, 200):
        print('\nEpoch: %d' % epoch)
        optimizer.param_groups = adjust_lr_steep(0.001, optimizer.param_groups, epoch, adj_lr)

        correct = 0
        total = 0
        loss_total=0
        pbar=tqdm(train_loader)
        for i, data in  enumerate(pbar):
            points, label = data
            points, label = points.to(device), label.to(device)[:,0]
            
            points = points.transpose(2, 1) # to be shape batch_size*3*N
            
            optimizer.zero_grad()

            pred, trans_feat = model(points)
                
            loss = criterion(pred, label)
            
            loss.backward()
            optimizer.step()
            pred_choice = pred.data.max(1)[1]
            correct += pred_choice.eq(label.data).cpu().sum()
            total += label.size(0)
            loss_total+=loss.item()
            #progress_bar(i, len(train_loader), 'Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'% (loss.item()/(i+1), 100.*correct.item()/total, correct, total))
           # s=str(i)+"   ,"+str(len(train_loader))+"  "+'Train Loss: %.3f | Train Acc: %.3f%% (%d/%d)'% (loss.item()/(i+1), 100.*correct.item()/total, correct, total)
            #pbar.set_description(s)
            pbar.set_postfix({'step':i+1,'Train Loss:' : total_loss/(i+1),'Train Acc':100.*correct.item()/total})
            
            
       
        train_loss, train_acc = loss_total/(i+1), 100.*correct.item()/total 
        ### Test in batch 
       
        test_loss, test_acc = test(model, test_loader, criterion)        
        acc_list.append(test_acc)  

        ### Keep tracing
        log_row(logname, [epoch, train_loss, train_acc, test_loss, test_acc, 
                optimizer.param_groups[0]['lr'], max(acc_list), np.argmax(acc_list)-1])
        save_ckpt( epoch, model, optimizer, acc_list)
        
       
        


Epoch: 0


KeyboardInterrupt: ignored